In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import log_loss

In [ ]:
Dtrain = xgb.DMatrix('Dataset/xgboost_train_visit.buffer')
Dtest = xgb.DMatrix('Dataset/xgboost_test_visit.buffer')

In [ ]:
Dtotal = xgb.DMatrix('Dataset/xgboost_total_visit.buffer')

### 1. Training

In [ ]:
evallist = [(Dtrain, 'train'),(Dtest, 'eval')]

In [ ]:
### change params here
import time
print("====================================")
print("model starts")
start = time.time()
model = xgb.train(params=params, 
    dtrain=Dtotal, 
    num_boost_round=100,  
    early_stopping_rounds=10, 
    evals=evallist)
score = log_loss(Dtotal.get_label(), model.predict(Dtotal))
end = time.time()
print("Time pass: ", end-start)
print("Parameters: ",params)
print("Score: ", score)

### 2. Final model

In [ ]:
params = {'eta': 0.35, 'max_depth': 9, 'num_class': 38, 'eval_metric': 'mlogloss', 'objective': 'multi:softprob'}

In [ ]:
import time
print("====================================")
print("model starts")
start = time.time()
model = xgb.train(params=params, 
    dtrain=Dtotal, 
    num_boost_round=100,  
    early_stopping_rounds=10, 
    evals=evallist)
score = log_loss(Dtotal.get_label(), model.predict(Dtotal))
end = time.time()
print("Time pass: ", end-start)
print("Parameters: ",params)
print("Score: ", score)

In [ ]:
model.save_model("Dataset/visit_" + str(params["eta"])+"_" + str(params["max_depth"])+"_100rounds.model")

### 3. Predict test data

In [ ]:
model = xgb.Booster(model_file = "Dataset/visit_gamma_0.35_9_0.05_20rounds.model") 

In [ ]:
real_test = xgb.DMatrix('Dataset/dtest_visit.buffer')

In [ ]:
predict_mat = model.predict(real_test)

In [ ]:
predict_mat.shape

### 4. Submit Kaggle

In [ ]:
import pandas as pd

In [ ]:
test = pd.read_csv('Dataset/clean_test_data.csv')

In [ ]:
result = pd.DataFrame(predict_mat)
dex = test.iloc[:,0]
submurge = pd.concat([dex,result], axis = 1)
print (submurge.shape)

In [ ]:
submurge.columns = ['VisitNumber', 'TripType_3','TripType_4','TripType_5','TripType_6','TripType_7',\
'TripType_8','TripType_9','TripType_12','TripType_14','TripType_15','TripType_18',\
'TripType_19','TripType_20','TripType_21','TripType_22','TripType_23','TripType_24',\
'TripType_25','TripType_26','TripType_27','TripType_28','TripType_29','TripType_30',\
'TripType_31','TripType_32','TripType_33','TripType_34','TripType_35','TripType_36',\
'TripType_37','TripType_38','TripType_39','TripType_40','TripType_41','TripType_42',\
'TripType_43','TripType_44','TripType_999']
submurge[['VisitNumber']] = submurge[['VisitNumber']].astype(int)
submurge.to_csv('Kaggle.csv', index = False, header=True)